
## Install Langchain packages


In [1]:
!pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.9 MB/s eta 0:00:00


In [2]:
# Installing openai's langchain libraries
!pip install langchain-openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [4]:
# Installing for vector store (embedding store)
!pip install faiss-cpu --quiet

# Importing Dependencies

In [5]:
import pandas as pd
import os.path as path
import os
import json
import getpass
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import PromptTemplate

# Importing Raw Dataset

In [52]:
# Specify the path to the Excel file
# file_path = 'sample_data/lawyers_dataset.xlsx'
file_path = 'sample_data/dataset1_lawyers.xlsx'

# Reading only first 100 rows from the Excel file into a DataFrame
# Limiting the dataset size imported in order to follow Company's directive to limit data published on GitHub
raw_df = pd.read_excel(file_path, nrows=100)

print("Dataset size = ", raw_df.shape)

# Print the first 5 rows of the DataFrame
raw_df.head()

Dataset size =  (100, 21)


,First Name,Last Name,Title,Company,Company Name for Emails,Seniority,Departments,# Employees,Industry,Keywords,...,City,State,Country,Company Address,Company City,Company State,Company Country,SEO Description,Technologies,Annual Revenue
0,Dennis,Brager,"Owner, Tax Litigation Attorney","Brager Tax Law Group, A P.C.",Brager Tax Law Group A P.C,Owner,"Finance, Legal",10,law practice,"tax litigation, tax controversy, tax evasion, ...",...,Los Angeles,California,United States,"11400 West Olympic Boulevard, Los Angeles, Cal...",Los Angeles,California,United States,All of our tax attorneys are ex-IRS trial lawy...,"Salesforce, EasyDNS, Outlook, Microsoft Office...",17760000.0
1,Charles,Spinner,Attorney/Owner,Spinner Law Firm P.A.,Spinner Law Firm P.A,Owner,Legal,7,law practice,NaN,...,Wesley Chapel,Florida,United States,"Wesley Chapel, Florida, United States",Wesley Chapel,Florida,United States,NaN,"Outlook, Microsoft Office 365, WordPress.org, ...",7000000.0
2,Caroline,Wang,"Corporate, Securities and M&A Lawyer, Founder ...",Wang Legal Professional Corporation,Wang Legal Professional,Founder,"C-Suite, Finance, Legal",2,law practice,"corporate finance, corporate & commercial law,...",...,Toronto,Ontario,Canada,"121 King St W, Toronto, Ontario, Canada, M5H 3T9",Toronto,Ontario,Canada,NaN,"Outlook, Google Cloud Hosting, Mobile Friendly...",NaN
3,Robert,Gaul,Subrogation Attorney and Founder,"Gaul & Associates, P.C.",Gaul & Associates P.C,Founder,"C-Suite, Legal",1,law practice,NaN,...,Philadelphia,Pennsylvania,United States,"1650 Arch St, Philadelphia, Pennsylvania, Unit...",Philadelphia,Pennsylvania,United States,NaN,"Gravity Forms, Bootstrap Framework, Facebook L...",1200000.0
4,Paul,Sheston,Founder/Lead Trial Attorney,"Sheston Law Group, PLLC",Sheston Law Group PLLC,Founder,"C-Suite, Legal",2,law practice,NaN,...,Scottsdale,Arizona,United States,"14550 N Frank Lloyd Wright Blvd, Scottsdale, A...",Scottsdale,Arizona,United States,"Sheston Law Group, PLLC is a boutique trial li...","Outlook, Microsoft Office 365, Google Cloud Ho...",NaN


In [53]:
# Taking a look at the dataset colummns
raw_df.columns

Index(['First Name', 'Last Name', 'Title', 'Company',
       'Company Name for Emails', 'Seniority', 'Departments', '# Employees',
       'Industry', 'Keywords', 'Website', 'City', 'State', 'Country',
       'Company Address', 'Company City', 'Company State', 'Company Country',
       'SEO Description', 'Technologies', 'Annual Revenue'],
      dtype='object')

In [54]:
# Taking a look at the dataset datatypes
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   First Name               100 non-null    object 
 1   Last Name                100 non-null    object 
 2   Title                    100 non-null    object 
 3   Company                  100 non-null    object 
 4   Company Name for Emails  100 non-null    object 
 5   Seniority                100 non-null    object 
 6   Departments              99 non-null     object 
 7   # Employees              100 non-null    int64  
 8   Industry                 100 non-null    object 
 9   Keywords                 53 non-null     object 
 10  Website                  100 non-null    object 
 11  City                     94 non-null     object 
 12  State                    97 non-null     object 
 13  Country                  100 non-null    object 
 14  Company Address          10

# Retriever LLM Set Up

In [55]:
# Loading API key to enable future OpenAI calls
open_ai_key = getpass.getpass()
os.environ["OPENAI_API_KEY"] = open_ai_key

··········


In [56]:
# Loading the ChatOpenAI model to be used
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [57]:
# Crafting the prompt to instruct the LLM model to perform specific desired tasks

prompt = PromptTemplate.from_template("""
You are a webscraper who can visit the url passed to you and also click on the links on the url page and fetch information in child pages.
Retrieve the response in a non-sentence JSON format. If the answer has more than one matches, please provide exact itemized list in non-sentence format.
If not answer found then respond with "0".

<context>
{context}
</context>

Question: {input}""")


#Creating a chain for passing a list of Documents to a model
document_chain = create_stuff_documents_chain(llm, prompt)

In [58]:
# LLM Loop Code Block

# Creating a list for storing the responses
response_list = []

# Creating a list to store the rows being read
rows_read = []

# Initiating the LLM
text_splitter = RecursiveCharacterTextSplitter()
embeddings = OpenAIEmbeddings()

# Looping through the dataframe to access the websites, load text from them, retrieve required data via LLM
# Storing data in dataframe at the end
for i in range(1, 20): ## CAN CHANGE SLICER HERE
    loader = WebBaseLoader(raw_df["Website"][i])
    docs = loader.load()
    documents = text_splitter.split_documents(docs)

    if len(documents) != 0:
        vector = FAISS.from_documents(documents, embeddings)
        retriever = vector.as_retriever()
        retrieval_chain = create_retrieval_chain(retriever, document_chain)

        response = retrieval_chain.invoke({"input": "Please click on url links to provide this firm's Services, Service fees, phone number from contact, email, customers, year founded and if it has a Client Login then yes or 0."})

        # Append the response to the list
        response_list.append(response["answer"])
        rows_read.append(i)

# Convert the list of responses to a DataFrame
data_dict_str = pd.DataFrame(response_list, columns=["answer"])

In [59]:
# Looking at the number of records collected by the LLM
len(data_dict_str)

18

In [60]:
# Taking a look at the data scraped by the LLM
data_dict_str

,answer
0,"{\n ""Services"": [\n ""Motor Vehicle C..."
1,"{\n ""Services"": ""Corporate and commercial la..."
2,"{\n ""Services"": ""Subrogation Law"",\n ""Se..."
3,"{\n ""Services"": [\n ""Business Immigr..."
4,"{\n ""Services"": [\n ""Personal Injury..."
5,"{\n ""Services"": [\n ""Foreclosure Def..."
6,"{\n ""Services"": [\n ""Automobile Acci..."
7,"{\n ""Services"": [\n ""Estate Planning..."
8,"{\n ""Services"": [\n ""Serious persona..."
9,"{\n ""Services"": [\n ""Personal Injury..."


# Data Transformation

In [62]:
# Fetching the exact same rows from raw_df as above LLM rows getting fetched
raw_df_to_concat = raw_df.loc[rows_read]
raw_df_to_concat

,First Name,Last Name,Title,Company,Company Name for Emails,Seniority,Departments,# Employees,Industry,Keywords,...,City,State,Country,Company Address,Company City,Company State,Company Country,SEO Description,Technologies,Annual Revenue
1,Charles,Spinner,Attorney/Owner,Spinner Law Firm P.A.,Spinner Law Firm P.A,Owner,Legal,7,law practice,NaN,...,Wesley Chapel,Florida,United States,"Wesley Chapel, Florida, United States",Wesley Chapel,Florida,United States,NaN,"Outlook, Microsoft Office 365, WordPress.org, ...",7000000.0
2,Caroline,Wang,"Corporate, Securities and M&A Lawyer, Founder ...",Wang Legal Professional Corporation,Wang Legal Professional,Founder,"C-Suite, Finance, Legal",2,law practice,"corporate finance, corporate & commercial law,...",...,Toronto,Ontario,Canada,"121 King St W, Toronto, Ontario, Canada, M5H 3T9",Toronto,Ontario,Canada,NaN,"Outlook, Google Cloud Hosting, Mobile Friendly...",NaN
3,Robert,Gaul,Subrogation Attorney and Founder,"Gaul & Associates, P.C.",Gaul & Associates P.C,Founder,"C-Suite, Legal",1,law practice,NaN,...,Philadelphia,Pennsylvania,United States,"1650 Arch St, Philadelphia, Pennsylvania, Unit...",Philadelphia,Pennsylvania,United States,NaN,"Gravity Forms, Bootstrap Framework, Facebook L...",1200000.0
5,Erick,Widman,Founder and Managing Attorney,Passage Immigration Law,Passage Immigration Law,Founder,"C-Suite, Legal",10,legal services,immigration,...,Portland,Oregon,United States,"810 NW Marshall St, Portland, Oregon, United S...",Portland,Oregon,United States,Discover Your Path to a New Beginning - Passag...,"Cloudflare DNS, Gmail, Google Apps, CloudFlare...",NaN
6,Tom,Mingus,Trial Attorney and Owner,Mingus Law Office,Mingus Law Office,Owner,Legal,10,law practice,NaN,...,Columbia,Missouri,United States,"11 N 6th St, Columbia, Missouri, United States...",Columbia,Missouri,United States,"If you need help with a personal injury, famil...","Google Font API, Cloudinary, YouTube, Google T...",927000.0
7,Siam,Joseph,Attorney / Owner,Law Office of Siam Joseph P.A.,Law Office of Siam Joseph P.A,Owner,Legal,4,law practice,NaN,...,NaN,NaN,United States,"618 U.S. 1, North Palm Beach, Florida, United ...",North Palm Beach,Florida,United States,NaN,"Gmail, Google Apps, GoDaddy Hosting, Apache, W...",NaN
8,Ronald,Crook,Attorney and Owner,Crook Law,Crook Law,Owner,Legal,1,law practice,"personal injury & social security disability, ...",...,Birmingham,Alabama,United States,"1100 Financial Ctr, Birmingham, Alabama, Unite...",Birmingham,Alabama,United States,Alabama law firm representing the seriously in...,"Outlook, Amazon AWS, Remote",NaN
9,Jonathan,Wade,Owner- Personal Injury Attorney,Wade Law Offices,Wade Law Offices,Owner,Legal,7,law practice,"business planning, tax controversy, succession...",...,Fayetteville,Georgia,United States,"2400 Professional Dr, Roseville, California, U...",Roseville,California,United States,NaN,"Sendgrid, Outlook, Amazon AWS, RingCentral, Go...",2709000.0
10,Michael,Serrano,Attorney/Owner,Serrano | McGuire,Serrano,Owner,Legal,3,law practice,"personal injury, employment law, civil litigat...",...,Lutz,Florida,United States,"25275 Wesley Chapel Blvd, Lutz, Florida, Unite...",Lutz,Florida,United States,If you or a loved one was injured in an accide...,"reCAPTCHA, WordPress.org, Google Tag Manager, ...",4477000.0
11,Frederick,Keck,Attorney/Owner,"Weilmuenster, Keck & Brown, P.C.",Weilmuenster Keck & Brown P.C,Owner,Legal,4,law practice,NaN,...,Belleville,Illinois,United States,"3201 W Main St, Belleville, Illinois, United S...",Belleville,Illinois,United States,NaN,"Apache, Google Analytics, Ubuntu, Remote, Vime...",19577000.0


In [64]:
# The number of records that were fetched by the LLM
print(len(response_list))

18


In [67]:
# Initializing an empty list to store indices
indices_to_keep = []

# Iterating through the DataFrame
for i, row in data_dict_str.iterrows():
    # Checking the condition
    if row['answer'] != '0':
        # Appending the index to the list
        indices_to_keep.append(i)

# Creating a new DataFrame containing only the rows that meet the condition
filtered_data_dict_str = data_dict_str.loc[indices_to_keep]

# Displaying the filtered DataFrame
filtered_data_dict_str

,answer
0,"{\n ""Services"": [\n ""Motor Vehicle C..."
1,"{\n ""Services"": ""Corporate and commercial la..."
2,"{\n ""Services"": ""Subrogation Law"",\n ""Se..."
3,"{\n ""Services"": [\n ""Business Immigr..."
4,"{\n ""Services"": [\n ""Personal Injury..."
5,"{\n ""Services"": [\n ""Foreclosure Def..."
6,"{\n ""Services"": [\n ""Automobile Acci..."
7,"{\n ""Services"": [\n ""Estate Planning..."
8,"{\n ""Services"": [\n ""Serious persona..."
9,"{\n ""Services"": [\n ""Personal Injury..."


In [68]:
# Iterating through the df and convert each row to a dictionary.

# Initializing a dictionary to store the converted rows
dict_values = []

for i, row in filtered_data_dict_str.iterrows():
      json_data = row['answer']
      data_dict_str2 = json.loads(json_data)
      # Iterate through each dictionary (row) and get the values for each row using list comprehension.
      row_values = [k for j, k in data_dict_str2.items()]
      # Append the list of values for each row to a list.
      dict_values.append(row_values)

In [69]:
# Creating a data DataFrame and adding each list of values, i.e., each row
# to the 'Services', 'Service fees', 'Phone number', 'Email', 'Customers', 'Year founded', 'Client Login'columns.

column_names = ['Services', 'Service fees', 'Phone number', 'Email', 'Customers', 'Year founded', 'Client Login']
data = pd.DataFrame(dict_values, columns=column_names)
print("Size of data", len(data))
data.head()

Size of data 18


,Services,Service fees,Phone number,Email,Customers,Year founded,Client Login
0,"[Motor Vehicle Cases, Personal Injury Cases, E...",Reasonable and clearly quoted at initial clien...,(813) 991-5099,Not provided,Individuals seeking legal assistance for perso...,Over 20 years ago,0
1,"Corporate and commercial law, securities law, ...",Not specified,(416) 546-4636,cwang@wanglegal.ca,Canadian and U.S. public and private companies...,Not specified,0
2,Subrogation Law,Not specified,Not provided,Not provided,Leading insurers,Not specified,0
3,"[Business Immigration, Family Immigration, Ind...","Exact, flat-fee quote provided during consulta...",(503) 427-8243,support@passage.law,250+ Happy Clients,2024,0
4,"[Personal Injury, Auto Accidents, Wrongful Dea...",Not specified on website,(573) 678-3105,Not provided on website,Individuals in need of legal services,Not specified on website,0


In [70]:
# Looking at the size of the new dataframe created
data.shape

(18, 7)

In [71]:
raw_df_to_concat.reset_index(drop = True)
raw_df_to_concat

,First Name,Last Name,Title,Company,Company Name for Emails,Seniority,Departments,# Employees,Industry,Keywords,...,City,State,Country,Company Address,Company City,Company State,Company Country,SEO Description,Technologies,Annual Revenue
1,Charles,Spinner,Attorney/Owner,Spinner Law Firm P.A.,Spinner Law Firm P.A,Owner,Legal,7,law practice,NaN,...,Wesley Chapel,Florida,United States,"Wesley Chapel, Florida, United States",Wesley Chapel,Florida,United States,NaN,"Outlook, Microsoft Office 365, WordPress.org, ...",7000000.0
2,Caroline,Wang,"Corporate, Securities and M&A Lawyer, Founder ...",Wang Legal Professional Corporation,Wang Legal Professional,Founder,"C-Suite, Finance, Legal",2,law practice,"corporate finance, corporate & commercial law,...",...,Toronto,Ontario,Canada,"121 King St W, Toronto, Ontario, Canada, M5H 3T9",Toronto,Ontario,Canada,NaN,"Outlook, Google Cloud Hosting, Mobile Friendly...",NaN
3,Robert,Gaul,Subrogation Attorney and Founder,"Gaul & Associates, P.C.",Gaul & Associates P.C,Founder,"C-Suite, Legal",1,law practice,NaN,...,Philadelphia,Pennsylvania,United States,"1650 Arch St, Philadelphia, Pennsylvania, Unit...",Philadelphia,Pennsylvania,United States,NaN,"Gravity Forms, Bootstrap Framework, Facebook L...",1200000.0
5,Erick,Widman,Founder and Managing Attorney,Passage Immigration Law,Passage Immigration Law,Founder,"C-Suite, Legal",10,legal services,immigration,...,Portland,Oregon,United States,"810 NW Marshall St, Portland, Oregon, United S...",Portland,Oregon,United States,Discover Your Path to a New Beginning - Passag...,"Cloudflare DNS, Gmail, Google Apps, CloudFlare...",NaN
6,Tom,Mingus,Trial Attorney and Owner,Mingus Law Office,Mingus Law Office,Owner,Legal,10,law practice,NaN,...,Columbia,Missouri,United States,"11 N 6th St, Columbia, Missouri, United States...",Columbia,Missouri,United States,"If you need help with a personal injury, famil...","Google Font API, Cloudinary, YouTube, Google T...",927000.0
7,Siam,Joseph,Attorney / Owner,Law Office of Siam Joseph P.A.,Law Office of Siam Joseph P.A,Owner,Legal,4,law practice,NaN,...,NaN,NaN,United States,"618 U.S. 1, North Palm Beach, Florida, United ...",North Palm Beach,Florida,United States,NaN,"Gmail, Google Apps, GoDaddy Hosting, Apache, W...",NaN
8,Ronald,Crook,Attorney and Owner,Crook Law,Crook Law,Owner,Legal,1,law practice,"personal injury & social security disability, ...",...,Birmingham,Alabama,United States,"1100 Financial Ctr, Birmingham, Alabama, Unite...",Birmingham,Alabama,United States,Alabama law firm representing the seriously in...,"Outlook, Amazon AWS, Remote",NaN
9,Jonathan,Wade,Owner- Personal Injury Attorney,Wade Law Offices,Wade Law Offices,Owner,Legal,7,law practice,"business planning, tax controversy, succession...",...,Fayetteville,Georgia,United States,"2400 Professional Dr, Roseville, California, U...",Roseville,California,United States,NaN,"Sendgrid, Outlook, Amazon AWS, RingCentral, Go...",2709000.0
10,Michael,Serrano,Attorney/Owner,Serrano | McGuire,Serrano,Owner,Legal,3,law practice,"personal injury, employment law, civil litigat...",...,Lutz,Florida,United States,"25275 Wesley Chapel Blvd, Lutz, Florida, Unite...",Lutz,Florida,United States,If you or a loved one was injured in an accide...,"reCAPTCHA, WordPress.org, Google Tag Manager, ...",4477000.0
11,Frederick,Keck,Attorney/Owner,"Weilmuenster, Keck & Brown, P.C.",Weilmuenster Keck & Brown P.C,Owner,Legal,4,law practice,NaN,...,Belleville,Illinois,United States,"3201 W Main St, Belleville, Illinois, United S...",Belleville,Illinois,United States,NaN,"Apache, Google Analytics, Ubuntu, Remote, Vime...",19577000.0


In [73]:
# Now modifying the above dataframe to have only those rows whose values are NOT zero
# raw_df_to_concat_final = raw_df_to_concat.loc[indices_to_keep]
raw_df_to_concat_final = raw_df_to_concat.loc[rows_read]
raw_df_to_concat_final

,First Name,Last Name,Title,Company,Company Name for Emails,Seniority,Departments,# Employees,Industry,Keywords,...,City,State,Country,Company Address,Company City,Company State,Company Country,SEO Description,Technologies,Annual Revenue
1,Charles,Spinner,Attorney/Owner,Spinner Law Firm P.A.,Spinner Law Firm P.A,Owner,Legal,7,law practice,NaN,...,Wesley Chapel,Florida,United States,"Wesley Chapel, Florida, United States",Wesley Chapel,Florida,United States,NaN,"Outlook, Microsoft Office 365, WordPress.org, ...",7000000.0
2,Caroline,Wang,"Corporate, Securities and M&A Lawyer, Founder ...",Wang Legal Professional Corporation,Wang Legal Professional,Founder,"C-Suite, Finance, Legal",2,law practice,"corporate finance, corporate & commercial law,...",...,Toronto,Ontario,Canada,"121 King St W, Toronto, Ontario, Canada, M5H 3T9",Toronto,Ontario,Canada,NaN,"Outlook, Google Cloud Hosting, Mobile Friendly...",NaN
3,Robert,Gaul,Subrogation Attorney and Founder,"Gaul & Associates, P.C.",Gaul & Associates P.C,Founder,"C-Suite, Legal",1,law practice,NaN,...,Philadelphia,Pennsylvania,United States,"1650 Arch St, Philadelphia, Pennsylvania, Unit...",Philadelphia,Pennsylvania,United States,NaN,"Gravity Forms, Bootstrap Framework, Facebook L...",1200000.0
5,Erick,Widman,Founder and Managing Attorney,Passage Immigration Law,Passage Immigration Law,Founder,"C-Suite, Legal",10,legal services,immigration,...,Portland,Oregon,United States,"810 NW Marshall St, Portland, Oregon, United S...",Portland,Oregon,United States,Discover Your Path to a New Beginning - Passag...,"Cloudflare DNS, Gmail, Google Apps, CloudFlare...",NaN
6,Tom,Mingus,Trial Attorney and Owner,Mingus Law Office,Mingus Law Office,Owner,Legal,10,law practice,NaN,...,Columbia,Missouri,United States,"11 N 6th St, Columbia, Missouri, United States...",Columbia,Missouri,United States,"If you need help with a personal injury, famil...","Google Font API, Cloudinary, YouTube, Google T...",927000.0
7,Siam,Joseph,Attorney / Owner,Law Office of Siam Joseph P.A.,Law Office of Siam Joseph P.A,Owner,Legal,4,law practice,NaN,...,NaN,NaN,United States,"618 U.S. 1, North Palm Beach, Florida, United ...",North Palm Beach,Florida,United States,NaN,"Gmail, Google Apps, GoDaddy Hosting, Apache, W...",NaN
8,Ronald,Crook,Attorney and Owner,Crook Law,Crook Law,Owner,Legal,1,law practice,"personal injury & social security disability, ...",...,Birmingham,Alabama,United States,"1100 Financial Ctr, Birmingham, Alabama, Unite...",Birmingham,Alabama,United States,Alabama law firm representing the seriously in...,"Outlook, Amazon AWS, Remote",NaN
9,Jonathan,Wade,Owner- Personal Injury Attorney,Wade Law Offices,Wade Law Offices,Owner,Legal,7,law practice,"business planning, tax controversy, succession...",...,Fayetteville,Georgia,United States,"2400 Professional Dr, Roseville, California, U...",Roseville,California,United States,NaN,"Sendgrid, Outlook, Amazon AWS, RingCentral, Go...",2709000.0
10,Michael,Serrano,Attorney/Owner,Serrano | McGuire,Serrano,Owner,Legal,3,law practice,"personal injury, employment law, civil litigat...",...,Lutz,Florida,United States,"25275 Wesley Chapel Blvd, Lutz, Florida, Unite...",Lutz,Florida,United States,If you or a loved one was injured in an accide...,"reCAPTCHA, WordPress.org, Google Tag Manager, ...",4477000.0
11,Frederick,Keck,Attorney/Owner,"Weilmuenster, Keck & Brown, P.C.",Weilmuenster Keck & Brown P.C,Owner,Legal,4,law practice,NaN,...,Belleville,Illinois,United States,"3201 W Main St, Belleville, Illinois, United S...",Belleville,Illinois,United States,NaN,"Apache, Google Analytics, Ubuntu, Remote, Vime...",19577000.0


In [74]:
# Concatenate the DataFrames vertically (along rows)

result_df = pd.concat([raw_df_to_concat_final.reset_index(drop=True), data], axis=1)

In [75]:
result_df.columns

Index(['First Name', 'Last Name', 'Title', 'Company',
       'Company Name for Emails', 'Seniority', 'Departments', '# Employees',
       'Industry', 'Keywords', 'Website', 'City', 'State', 'Country',
       'Company Address', 'Company City', 'Company State', 'Company Country',
       'SEO Description', 'Technologies', 'Annual Revenue', 'Services',
       'Service fees', 'Phone number', 'Email', 'Customers', 'Year founded',
       'Client Login'],
      dtype='object')

In [79]:
result_df.shape

(18, 28)

In [78]:
result_df.head()

,First Name,Last Name,Title,Company,Company Name for Emails,Seniority,Departments,# Employees,Industry,Keywords,...,SEO Description,Technologies,Annual Revenue,Services,Service fees,Phone number,Email,Customers,Year founded,Client Login
0,Charles,Spinner,Attorney/Owner,Spinner Law Firm P.A.,Spinner Law Firm P.A,Owner,Legal,7,law practice,NaN,...,NaN,"Outlook, Microsoft Office 365, WordPress.org, ...",7000000.0,"[Motor Vehicle Cases, Personal Injury Cases, E...",Reasonable and clearly quoted at initial clien...,(813) 991-5099,Not provided,Individuals seeking legal assistance for perso...,Over 20 years ago,0
1,Caroline,Wang,"Corporate, Securities and M&A Lawyer, Founder ...",Wang Legal Professional Corporation,Wang Legal Professional,Founder,"C-Suite, Finance, Legal",2,law practice,"corporate finance, corporate & commercial law,...",...,NaN,"Outlook, Google Cloud Hosting, Mobile Friendly...",NaN,"Corporate and commercial law, securities law, ...",Not specified,(416) 546-4636,cwang@wanglegal.ca,Canadian and U.S. public and private companies...,Not specified,0
2,Robert,Gaul,Subrogation Attorney and Founder,"Gaul & Associates, P.C.",Gaul & Associates P.C,Founder,"C-Suite, Legal",1,law practice,NaN,...,NaN,"Gravity Forms, Bootstrap Framework, Facebook L...",1200000.0,Subrogation Law,Not specified,Not provided,Not provided,Leading insurers,Not specified,0
3,Erick,Widman,Founder and Managing Attorney,Passage Immigration Law,Passage Immigration Law,Founder,"C-Suite, Legal",10,legal services,immigration,...,Discover Your Path to a New Beginning - Passag...,"Cloudflare DNS, Gmail, Google Apps, CloudFlare...",NaN,"[Business Immigration, Family Immigration, Ind...","Exact, flat-fee quote provided during consulta...",(503) 427-8243,support@passage.law,250+ Happy Clients,2024,0
4,Tom,Mingus,Trial Attorney and Owner,Mingus Law Office,Mingus Law Office,Owner,Legal,10,law practice,NaN,...,"If you need help with a personal injury, famil...","Google Font API, Cloudinary, YouTube, Google T...",927000.0,"[Personal Injury, Auto Accidents, Wrongful Dea...",Not specified on website,(573) 678-3105,Not provided on website,Individuals in need of legal services,Not specified on website,0


In [80]:
# Making a copy of the result_df for the first chunk of data obtained; Deep copy
result_df = result_df.copy(deep=True)

In [81]:
result_df.to_csv('sample_data/result_df.csv', index=False)

### Summary

In this  code file, the original dataset obtained form the company is being imported into a dataframe. That data is then utilized to feed the Langchain LLM with the websites from the dataframe sequentially looping through it. The LLM utilizes the text scraping abilities and the directions from the encoded prompt above to fetch target data from the websites. The data obtained is then transformed and stored into a csv file for further pre-processing and cleaning in the Data Cleaning code file.